In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [3]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [6]:
def train_model_by_X_y(_X, _y, iterations=5000):
    skf = KFold(n_splits=5, random_state=2048, shuffle=True)
    skf.get_n_splits(_X, _y)

    train_index_list = []
    test_index_list = []
    train_x_list = []
    val_x_list = []
    train_y_list = []
    val_y_list = []

    for i, (train_index, test_index) in enumerate(skf.split(_X, _y)):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

        train_x_list.append(_X[train_index, :])
        val_x_list.append(_X[test_index, :])
        train_y_list.append(_y[train_index])
        val_y_list.append(_y[test_index])    
   

    cat_models = []
    for i in range(5):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i],
        clf = CatBoostRegressor(iterations=iterations, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=5000, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models


In [21]:
def cal_train_pred(models, _X):
    preds = []
    for model in models:
        preds.append(model.predict(_X))
    s = 0
    for pred in preds:
        s += pred
    s = s/5
    return s

In [49]:
# y = train_df[target[0]].values
# X = train_df[main_attrs].values
# cat_models = train_model_by_X_y(X, y)

In [51]:
check_col = 'contest-wind-h500-14d__wind-hgt-500'

In [54]:
train_df.groupby(['lat', 'lon'])[check_col].mean().reset_index()

,lat,lon,contest-wind-h500-14d__wind-hgt-500
0,0.000000,0.833333,5856.087989
1,0.000000,0.866667,5855.675951
2,0.045455,0.833333,5844.286566
3,0.045455,0.866667,5845.395157
4,0.045455,0.900000,5846.219439
...,...,...,...
509,1.000000,0.733333,5594.640109
510,1.000000,0.766667,5593.227921
511,1.000000,0.800000,5579.298919
512,1.000000,0.833333,5581.151382
